# Leadville Results Analysis

First, let's import the packages that will be used throughout this analysis.

In [1]:
import pandas as pd

from processing import cleaners, io, util

RACE_YEAR = 2019

Then change a setting so we can see the results of variable assignments.

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr_or_assign'

This notebook assumes we have already run a scrapy spider to collect race data like so:

```python
from processing.scrapers import scrape_for_pandas
scrape_for_pandas(RACE_YEAR)
```

## Load, process, and clean split info scraped by the spider.

### Load the data (as scraped by the spider) into a pandas DataFrame.

In [3]:
df_split_info_raw = io.load_df_split_info_raw(RACE_YEAR)

,distance_m
name,
Full Course,161578
12.6mi_Start to May Queen,20278
23.5mi_Start to Outward Bound OUT,37820
29.3mi_Start to Half Pipe Out,47154
37.9mi_Start to Twin Lakes Out,60994
43.5mi_Start to Hope Pass Out,70006
50mi_Start to Winfield,80467
56.5mi_Start to Hope Pass In,90928
62.5mi_Start to Twin Lakes In,100584


In [4]:
df_split_info = cleaners.process_df_split_info(df_split_info_raw)

# Debug
# df_split_info
# df_splits.dtypes

,distance_m,distance_mi
label,,
12.6mi_Start to May Queen,20278,12.6
23.5mi_Start to Outward Bound OUT,37820,23.5
29.3mi_Start to Half Pipe Out,47154,29.3
37.9mi_Start to Twin Lakes Out,60994,37.9
43.5mi_Start to Hope Pass Out,70006,43.5
50mi_Start to Winfield,80467,50.0
56.5mi_Start to Hope Pass In,90928,56.5
62.5mi_Start to Twin Lakes In,100584,62.5
71.1mi_Start to Half Pipe In,114424,71.1


### Apply some Leadville-specific processing to the split info

In [5]:
# (present in 2021, gone in 2022, and never very useful)
df_split_info.drop(index='Kick to Finish 0.8 Miles Left', errors='ignore', inplace=True)

# Extract each aid station name from its 'label' string.
# NOTE: This isn't really necessary, just aesthetic. Move to plotting script?
df_split_info['name'] = df_split_info.index.to_series(
  ).str.extract(r'Start to (.+)', expand=False,
  # If the split name doesn't match the regex, just use as-is.
  ).fillna(df_split_info.index.to_series())

# Manually add cutoff info (found elsewhere on the web)
df_split_info['cutoff_hr'] = [3.75, 6., 7.5, 9.5, 12., 14., None, 18., 21.25, 23., 26.5, 30.]

df_split_info

# Debug
# df_splits.dtypes
# type(df_splits['cutoff_hr'].iloc[6])  # np.float64

,distance_m,distance_mi,name,cutoff_hr
label,,,,
12.6mi_Start to May Queen,20278,12.6,May Queen,3.75
23.5mi_Start to Outward Bound OUT,37820,23.5,Outward Bound OUT,6.00
29.3mi_Start to Half Pipe Out,47154,29.3,Half Pipe Out,7.50
37.9mi_Start to Twin Lakes Out,60994,37.9,Twin Lakes Out,9.50
43.5mi_Start to Hope Pass Out,70006,43.5,Hope Pass Out,12.00
50mi_Start to Winfield,80467,50.0,Winfield,14.00
56.5mi_Start to Hope Pass In,90928,56.5,Hope Pass In,NaN
62.5mi_Start to Twin Lakes In,100584,62.5,Twin Lakes In,18.00
71.1mi_Start to Half Pipe In,114424,71.1,Half Pipe In,21.25


### (Optional) write cleaned split info to file

```python
import os

# TODO: implement this
# io.save_df_split_info(df_split_info, RACE_YEAR)

# dir_out = settings.CLEAN_RACE_DATA_DIR
dir_out = io.get_clean_race_data_dir(RACE_YEAR)
if not os.path.exists(dir_out):
  os.makedirs(dir_out)

# df_split_info.to_csv(os.path.join(dir_out, settings.SPLIT_INFO_FNAME),
df_split_info.to_csv(os.path.join(dir_out, io.SPLIT_INFO_FNAME),
  columns=['name', 'distance_mi', 'cutoff_hr'],
  index=True)
```

## Load, process, and clean the athlete split data scraped by the spider

The Athlinks spider saves split data for each athlete. Load all such split data from the spider-produced `json` file into a `pandas.DataFrame`.

In [6]:
df_split_times_raw = io.load_df_split_times_raw(RACE_YEAR)

util.df_td_fmt(df_split_times_raw)

,Anton Samokhvalov,Justin Ladner,Jeffrey Urbanski,Radu Milea,steve slauson,Daniel Frank,Clark Messman,RUBEN DELGADO GIL,Cat Bradley,Max Robinson,...,Bruno Furrer,Alexandra Mittelman,Donna Provenzano,Jeremiah Jenson,Denzil Jennings,Becky Grebosky,Trey Simcik,Joe Miller,Brian Burton,Ryan McAfee
Full Course,22:01:29,21:04:42,21:12:28,21:28:03,21:35:52,21:58:23,21:15:31,21:00:20,20:45:48,20:51:47,...,29:02:07,28:59:38,28:58:56,29:00:31,28:59:04,28:58:26,28:57:40,28:46:50,28:56:50,28:57:30
12.6mi_Start to May Queen,01:42:57,01:42:33,01:54:28,01:50:09,01:43:05,01:57:18,01:50:40,01:43:14,01:50:54,01:49:21,...,02:25:26,02:28:19,02:31:08,02:15:38,02:24:07,02:37:37,02:07:55,02:22:42,02:37:10,02:20:37
23.5mi_Start to Outward Bound OUT,03:25:59,03:26:33,03:45:52,03:40:46,03:31:29,03:47:47,03:38:10,03:29:35,03:43:02,03:31:07,...,05:01:49,04:54:48,05:01:23,04:43:35,04:49:11,05:05:12,04:14:36,04:50:31,05:04:30,04:38:31
29.3mi_Start to Half Pipe Out,04:17:12,04:17:33,04:39:39,04:34:48,04:27:23,04:42:29,04:29:34,04:21:32,05:01:23,04:17:37,...,06:25:16,06:06:49,06:22:50,06:03:59,06:06:06,06:20:26,05:17:12,06:03:53,06:18:31,05:49:20
37.9mi_Start to Twin Lakes Out,05:59:39,05:42:44,06:10:57,06:11:01,06:01:49,06:16:30,05:54:20,05:53:35,06:08:34,05:41:57,...,08:37:40,08:10:06,08:31:29,08:10:46,08:12:13,08:28:14,07:03:35,08:05:03,08:19:02,07:47:05
43.5mi_Start to Hope Pass Out,07:58:18,07:33:02,08:00:26,07:59:41,08:00:12,08:17:31,07:49:04,07:49:31,07:55:23,07:20:59,...,11:18:31,10:51:54,11:14:13,10:43:53,10:58:53,11:14:04,09:40:02,10:34:31,11:04:33,10:19:29
50mi_Start to Winfield,09:08:37,08:50:00,09:19:39,09:15:27,09:18:50,09:36:05,09:10:43,09:04:26,09:07:32,08:36:49,...,13:02:31,12:42:50,13:06:40,12:37:09,12:44:57,13:06:40,11:21:32,12:23:59,12:43:38,12:00:42
56.5mi_Start to Hope Pass In,11:20:27,10:48:53,11:21:07,11:09:56,11:14:42,11:41:16,11:28:39,11:09:08,11:03:52,10:42:53,...,15:31:31,15:25:57,15:36:48,15:14:10,15:20:56,15:47:36,14:14:32,15:02:02,15:28:22,14:43:50
62.5mi_Start to Twin Lakes In,12:34:52,12:11:28,12:46:14,12:32:04,12:27:58,12:57:45,13:04:04,12:27:43,12:15:49,12:11:25,...,17:21:10,17:29:07,17:25:49,17:11:21,17:20:54,17:47:07,16:06:14,17:05:49,17:33:52,16:45:18
71.1mi_Start to Half Pipe In,14:22:52,14:01:03,14:39:38,14:21:35,14:27:36,14:49:31,14:40:29,14:16:41,14:22:47,14:00:47,...,19:52:57,<NA>,20:15:16,19:38:44,19:51:13,20:24:17,18:48:06,19:42:52,19:54:34,19:28:41


Notice that the columns (athletes) are presented in the order the Athlinks website spider returned them. This isn't particularly useful for spotting outliers or unreasonable performances. It would be easier to spot bad data if athletes with similar performances were grouped together. To fix this, we can group the athletes based on the furthest split they each recorded, then sort athletes within each group by their split times.

In [7]:
# Load the split info that's been processed and cleaned
# NOTE: no need to reload, I created it above.
# df_split_info = io.load_df_split_info_clean(settings.CLEAN_RACE_DATA_DIR)
# print(df_split_info_clean)

# df_split_times = util.load_athlete_split_times(clean=False)
# df_split_times = cleaners.sort_df_split_data(
#   io.load_df_split_times_raw(settings.RAW_RACE_DATA_DIR),
#   df_split_info_clean)

df_split_times = cleaners.sort_df_split_data(df_split_times_raw, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

Charles Corfield Ryan Smith Chad Trammell Devon Olson Ramon Casanovas Tate Knight Mark Marzen Gediminas Grinius Jock Green Gabe Joyes Matthew Urbanski Magdalena Boulet Vineer Bhansali Christopher DeNucci Rodrigo Jimenez Cat Bradley Max Robinson RUBEN DELGADO GIL Justin Ladner Marvin Sandoval Jeffrey Urbanski Clark Messman Radu Milea steve slauson Daniel Frank Anton Samokhvalov Jeremy Bradford Jacob Banta Lance Anderson Jon Harrison Peter Mortimer  Pat Cade Matthew Harrison Wesley Sandoval Mackenzie Mau Jamie Kilcoyne David Goggins Ryan Van Ness Chad Braun Jonathan Lantz TODD DILL Bryan Kerl Adam Monke Nathaniel Orders Tito Nazar Ezra Becker Zachary Allen Kilian Korth Brooks Williams John Olar Brandon Worthington Samantha Wood Carrie Stafford James Schneiderman Bruce Dailey Dustin Moore Jordan Ricks Harry Harcrow Jared Conlin Timothy Barr Willie Stewart Chavet Breslin Tim Finocchio Sacha Devillaz Phil Reutlinger Drew Adams John Muir Max Fulton paul zani Lelis Gonzalez Brandon Ray Steve Orellana Jens Sperlich Alex Cummings matt harvey Steve Pentler Howard Vogt Daniel Arnold Sam Schwaller Aaron Acuna Leonardo Leite Tim Cronk Michael Robbert Robert Stowe Travis Daniels Niki Pardoe ANDREW WICKLESS jason vieth Michael Alfred Kaitlyn Morgan George Ulrich Michael King Dion Leonard Aaron Ellison Joshua Stevens Craig Sylvester Shannon Howell Chris Wright kenneth ebener RANDY MAST Shaun Baker Gary Stotler aaron fearing Nikita Levchenko Kara Diamond-Husmann David Mackey Jonathan Joslyn Paul Tucker Mat Grills Stacey Marion Peter Grenney Jason Romero Francesc Basacoma Matthew Hunsucker Janis Snepste Ondrej Tomek Richie Dart Neil Feldman Benjamin Hauschulz Chris Mead Rod Bolls Emily Messegee Joshua Dickson Hannah Jochem Katrin Silva MELANIE WHITE kenneth erichsen Stephen Rodgers Giles Hedley John Mollenhour Daniela Seyler Walter Handloser Bill Hoffman Alan Smith Cristobal Forno David Kerr III Clayton Shapiro Dan Zolnikov Christine O'Gorman Jeremy Howard Hope Bain Tracy O'Brien Anders Wood John Goble Randy Semrau Jay Hagan Emily Luhrs Cristian Vargas Max Lawler Nic De Beer Jeff Mohrmann Sean McLaughlin Rob Wright Katy Anderson Amar Kuchinad Cory Leppert Matthew Kirkendall Dreama Walton Wade Jarvis Max Forgensi Tom Welker Andrew Flinn Drew McManus Andy Bourke Jeff Zenger Martin Schneekloth R. Sean Churchill Michael Nothem Michael Sylvester Gabriel Hendry Erin Park Lillie Romeiser Zach Allen Lucas Onan Andrew Coleman Luke Pospisil Eric Nelson Robert Wright Brandon Wilmoth Denise McHale javier vilchis Clark Mitchell Caitlin Barr Neal Palles April Kenyon Marc Grabowski Jemil Kassahun Kenneth Biddison Erin Drasler John (Muck) kilpatrick TR Maloney Carey Willson LaMar Jost John Paul Ogden Lance Pearce Pete Kardasis Phil Atkinson Ted Richardson John Music Sonja Brester Robert J Koch Mike Wasson Geoff Hanley Canice Harte BAREFOOT MCDONALD Scott Sundahl James Koch Brandy Taylor Jason Goss Joe Miller Paul Dube Newton McCollum tyler quinn Kayde Anderson Lisa Nelson Andrew Erickson Mihai Serban ROBERT MAZURKIEWICZ helen degennaro Caspar Sprungli Brian Burton Ryan McAfee Trey Simcik Becky Grebosky Donna Provenzano Denzil Jennings Alexandra Mittelman Jeremiah Jenson Sam Stephenson Bruno Furrer Amanda Webb Neil Smith Colleen Mentz Chris Kumm Jason Carey Brad Anderson Ciro Ortega Andrew Gzybowski Madeline Hirschfeld Jeremiah Nemechek Erik Nielsen Moon-Ka LEUNG Jack Repasky Erik Hull Brian Merseal Dave Ballard Kevin Moore Erin Laplander Charles Bybee Walter Edwards Kevin Koncilja Jimmy Daly Brian Passenti Abby Reinholt Evan Weisel Junko Kazukawa Bryan Wisdom Allen Kinsler Trish Holbel Sergio Saenz Robb McLean RAMIRO HECTOR CALDERON  COLOMO Paul Nielsen Chung-Man Tam Tyler Patterson Ted Politte Ethan Pence Todd Pizzi Sean Doran Monica Obsitos Jacey Anderson Karolina Zavisiute Jim Ploof Jack Rawls Aaron Cook Benjamin Corwin Luke Finken Alexa Roop Stephen Brown Brent Heady Kristin Walls Daniel Madalena Hawaiian Shirt Ray Churgovich Nathan Pierce Ben Santerre Har

In [8]:

# Optional: Print to a file so
# I can inspect the whole thing for abnormalities (alt+z to disable wrap).
# Mostly looking for anomalous finish line splits when a runner DNFs and 
# returns to Leadville.
# NOTE: Cannot view this in vscode if it goes over 10k characters wide.
# Sublime text can do it.
# util.full_df_to_file(
#   util.df_td_fmt(df_split_times),
#   os.path.join(settings.RAW_RACE_DATA_DIR, 'df_all.txt')
# )

### Remove wonky runner data, as determined by manual inspection.

Viewing the data this way highlights erroneous split data. For example, look at the first column, containing the splits for Charles Corfield. The uncleaned split data suggests that Charles won the entire race, because his finish line split is the fastest. But looking more closely, it seems he dropped out of the race after the split labeled `56.5mi_Start to Hope Pass In`, and his bib triggered the sensor at the finish line when he came to watch the 2019 champion, Ryan Smith, bring it home. Something similar seemed to happen with Vineer Bhansali.

In [9]:
util.df_td_fmt(
  df_split_times[['Charles Corfield', 'Ryan Smith', 'Vineer Bhansali']]
)

,Charles Corfield,Ryan Smith,Vineer Bhansali
label,,,
12.6mi_Start to May Queen,02:25:16,01:36:52,02:32:48
23.5mi_Start to Outward Bound OUT,04:37:43,03:10:01,05:15:49
29.3mi_Start to Half Pipe Out,05:44:52,03:56:17,06:38:03
37.9mi_Start to Twin Lakes Out,07:35:51,05:14:23,08:52:44
43.5mi_Start to Hope Pass Out,10:05:19,06:46:09,12:03:43
50mi_Start to Winfield,11:35:04,07:47:15,13:58:44
56.5mi_Start to Hope Pass In,13:56:55,09:22:44,<NA>
62.5mi_Start to Twin Lakes In,<NA>,10:17:55,<NA>
71.1mi_Start to Half Pipe In,<NA>,11:49:43,<NA>


To cleanse the data of these erroneous splits, we just need to set them to a null value.

In [10]:
df_split_times.loc['Full Course', 'Charles Corfield'] = pd.NA
df_split_times.loc['Full Course', 'Vineer Bhansali'] = pd.NA

util.df_td_fmt(
  df_split_times[['Charles Corfield', 'Ryan Smith', 'Vineer Bhansali']]
)

,Charles Corfield,Ryan Smith,Vineer Bhansali
label,,,
12.6mi_Start to May Queen,02:25:16,01:36:52,02:32:48
23.5mi_Start to Outward Bound OUT,04:37:43,03:10:01,05:15:49
29.3mi_Start to Half Pipe Out,05:44:52,03:56:17,06:38:03
37.9mi_Start to Twin Lakes Out,07:35:51,05:14:23,08:52:44
43.5mi_Start to Hope Pass Out,10:05:19,06:46:09,12:03:43
50mi_Start to Winfield,11:35:04,07:47:15,13:58:44
56.5mi_Start to Hope Pass In,13:56:55,09:22:44,<NA>
62.5mi_Start to Twin Lakes In,<NA>,10:17:55,<NA>
71.1mi_Start to Half Pipe In,<NA>,11:49:43,<NA>


Another type of error can happen when a split is triggered erroneously early, for whatever reason. This type of erroneous data doesn't necessarily jump out upon visual inspection. One way to check for it is to see if any athletes data includes a negative point-to-point time.

In [11]:
# df_split_times.diff().iloc[1:].apply(lambda col: (col.dt.total_seconds() < 0).any())
# df_split_times.diff().iloc[1:].apply(lambda col: col[~col.isnull()].dt.total_seconds() < 0).all()
# df_split_times.diff().iloc[1:].apply(lambda col: (col.dt.total_seconds() > 0))
# df_split_times.diff().iloc[1:].apply(lambda col: (col.dt.total_seconds() > 0).all())
# df_split_times.apply(lambda col: col.is_monotonic_increasing)

# series_athlete_has_negative_timedelta = df_split_times.diff().apply(
#   lambda col: (col.dt.total_seconds() < 0).any())

series_athlete_has_negative_timedelta = df_split_times.apply(
  lambda col: (col[col.notnull()].dt.total_seconds().diff() < 0).any())

# series_athlete_has_negative_timedelta = df_split_times.apply(
#   lambda col: col.notnull()
# )

util.df_td_fmt(
  df_split_times.loc[:, series_athlete_has_negative_timedelta]
)

,Willie Stewart,Max Fulton
label,,
12.6mi_Start to May Queen,02:37:41,02:07:35
23.5mi_Start to Outward Bound OUT,05:05:51,<NA>
29.3mi_Start to Half Pipe Out,06:25:48,05:19:12
37.9mi_Start to Twin Lakes Out,08:34:21,<NA>
43.5mi_Start to Hope Pass Out,10:52:02,12:51:24
50mi_Start to Winfield,12:36:47,10:34:47
56.5mi_Start to Hope Pass In,15:13:10,<NA>
62.5mi_Start to Twin Lakes In,17:25:27,14:26:52
71.1mi_Start to Half Pipe In,20:55:16,<NA>


It's not immediately clear what is going on with the split data for either of these athletes.

In Willie's case, the erroneously negative elapsed time comes between the splits at 76.9 miles and 87.8 miles. But even if we discarded the split data at 87.8 miles as an obvious misfire, no one could cover the distance from 76.9 miles to the finish in such a short time. But is the error at 76.9 miles or the finish line? Both?

It's impossible to tell which split data is valid or invalid without more context. Let's look at athletes with similar finish times to each athlete with bad data.

In [12]:
ix = df_split_times.columns.get_loc('Willie Stewart')
util.df_td_fmt(
  df_split_times.iloc[:, ix-5:ix+5]
)

,Dustin Moore,Jordan Ricks,Harry Harcrow,Jared Conlin,Timothy Barr,Willie Stewart,Chavet Breslin,Tim Finocchio,Sacha Devillaz,Phil Reutlinger
label,,,,,,,,,,
12.6mi_Start to May Queen,01:54:36,01:53:51,02:10:23,02:10:16,01:57:09,02:37:41,02:27:43,02:11:46,01:48:54,01:45:33
23.5mi_Start to Outward Bound OUT,03:46:50,03:50:14,04:25:07,04:23:24,03:58:18,05:05:51,05:01:15,04:26:39,03:42:10,03:27:17
29.3mi_Start to Half Pipe Out,04:41:33,04:49:37,05:36:59,05:33:02,05:03:23,06:25:48,06:15:07,05:33:07,04:44:40,04:18:51
37.9mi_Start to Twin Lakes Out,06:08:45,06:34:31,07:30:58,07:25:00,06:48:51,08:34:21,08:18:11,07:29:20,06:36:00,05:49:22
43.5mi_Start to Hope Pass Out,07:58:00,08:39:24,10:05:20,09:55:27,08:53:27,10:52:02,11:18:24,09:38:59,08:40:43,07:45:10
50mi_Start to Winfield,09:10:02,10:03:20,11:32:32,11:28:55,10:15:48,12:36:47,13:29:12,11:10:27,10:06:14,09:07:02
56.5mi_Start to Hope Pass In,11:09:45,12:10:13,14:15:05,13:55:57,12:21:24,15:13:10,15:59:55,13:32:45,12:24:45,11:23:10
62.5mi_Start to Twin Lakes In,12:21:07,13:43:02,16:35:13,15:49:43,13:52:13,17:25:27,<NA>,15:01:03,13:46:35,13:10:05
71.1mi_Start to Half Pipe In,14:14:20,15:42:24,19:17:30,19:16:10,15:49:39,20:55:16,21:18:09,17:04:44,15:50:11,15:18:17


Wow now I have no idea what data is valid. 
* It seems likely that Willie Stewart DNFed at 76.9. I just doubt anyone could cover 71.1 -> 100 in 3ish hours,
  so it seems very unlikely that the split at 76.9 is a misfire.
* Chavet Breslin probably DNFed at 76.9 and recorded an erroneous finish.
* Harry Harcrow and Jared Conlin seem to have DNFed at 71.1 but recorded a finish line split near each other.
* Tim Finocchio seemed sketchy at first with how much ground he made up in the later splits, but I
  cannot say anything without busting out the statistics.

Let's cleanse the more likely errors.

In [13]:
df_split_times.loc['87.8mi_Start to May Queen In': 'Full Course', 'Willie Stewart'] = pd.NA
df_split_times.loc['Full Course', 'Chavet Breslin'] = pd.NA
df_split_times.loc['Full Course', 'Harry Harcrow'] = pd.NA
df_split_times.loc['Full Course', 'Jared Conlin'] = pd.NA

Let's look at the context around Max's finish split.

In [14]:
ix = df_split_times.columns.get_loc('Max Fulton')
util.df_td_fmt(
  df_split_times.iloc[:, ix-5:ix+5]
)

,Tim Finocchio,Sacha Devillaz,Phil Reutlinger,Drew Adams,John Muir,Max Fulton,paul zani,Lelis Gonzalez,Brandon Ray,Steve Orellana
label,,,,,,,,,,
12.6mi_Start to May Queen,02:11:46,01:48:54,01:45:33,02:05:26,01:57:09,02:07:35,02:13:43,01:55:02,01:58:26,01:57:08
23.5mi_Start to Outward Bound OUT,04:26:39,03:42:10,03:27:17,04:09:53,03:52:19,<NA>,<NA>,03:48:48,<NA>,03:50:59
29.3mi_Start to Half Pipe Out,05:33:07,04:44:40,04:18:51,05:13:16,04:52:39,05:19:12,05:20:37,04:45:57,05:09:08,04:56:55
37.9mi_Start to Twin Lakes Out,07:29:20,06:36:00,05:49:22,07:06:31,06:38:57,<NA>,07:04:58,06:27:16,06:59:22,06:37:55
43.5mi_Start to Hope Pass Out,09:38:59,08:40:43,07:45:10,09:14:54,08:49:31,12:51:24,09:07:22,08:22:41,09:18:31,08:53:13
50mi_Start to Winfield,11:10:27,10:06:14,09:07:02,10:45:13,10:12:16,10:34:47,10:50:19,09:54:44,10:45:07,10:25:28
56.5mi_Start to Hope Pass In,13:32:45,12:24:45,11:23:10,13:12:36,12:33:15,<NA>,13:02:51,11:53:52,12:59:48,12:52:42
62.5mi_Start to Twin Lakes In,15:01:03,13:46:35,13:10:05,14:43:07,14:05:07,14:26:52,14:45:50,13:36:44,14:29:31,14:25:16
71.1mi_Start to Half Pipe In,17:04:44,15:50:11,15:18:17,16:51:55,16:14:47,<NA>,16:47:20,15:51:56,16:33:59,16:33:57


Immediately, it looks like the split at 43.5 misfired. I think his chip didn't fire on the way out,
but it did fire on the way in. Let's swap those values.

In [15]:
df_split_times.loc['56.5mi_Start to Hope Pass In', 'Max Fulton'] = df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Max Fulton']
df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Max Fulton'] = pd.NA

In [16]:

# TODO: Use statistical methods or reasonable assumptions to 
# programmatically flag sketchy split times. Single splits that are
# anomalously fast are invisible to me when I'm visually checking the
# output.

# TODO: Move the functionality below somewhere else - it represents insights
#       that can only be gained after looking at the results of the
#       script so far.

# TODO: Make this programmatic if it needs to be done more than once.
if RACE_YEAR == 2022:
  # Erroneous finish line split after a dnf
  df_split_times.loc['Full Course', 'Mickey Davis'] = pd.NA

  df_split_times.loc['76.9mi_Start to Outward Bound In', 'Timothy Weng'] = pd.NA

  # He would have had to run an unreasonably fast time to get to the next split. 
  df_split_times.loc['29.3mi_Start to Half Pipe Out', 'Adrian Macdonald'] = pd.NA

  # chip reported a negative time from last split
  df_split_times.loc['43.5mi_Start to Hope Pass Out', 'Kris Rugloski'] = pd.NA

  # 05:10:20 split would put him in first. Hmm.
  # Next two splits are pretty fast for his eventual finish time too.
  # Could have been a wonky chip or an aggressive strategy, idk.
  # df_split_times.loc['37.9mi_Start to Twin Lakes Out', 'Michael Nanaszko'] = pd.NA

elif RACE_YEAR == 2021:
  # chip reported a negative time from last split
  df_split_times.loc['37.9mi_Start to Twin Lakes Out', 'Matthew DuBois'] = pd.NA
  # TODO: finish checking.

df_split_times = cleaners.sort_df_split_data(df_split_times, df_split_info)

util.display_full_df(util.df_td_fmt(df_split_times))

# series_athlete_has_negative_timedelta = df_split_times.apply(
#   lambda col: (col[col.notnull()].dt.total_seconds().diff() < 0).any())
# df_split_times.loc[:, series_athlete_has_negative_timedelta]

Ryan Smith Chad Trammell Devon Olson Ramon Casanovas Tate Knight Mark Marzen Gediminas Grinius Jock Green Gabe Joyes Matthew Urbanski Magdalena Boulet Christopher DeNucci Rodrigo Jimenez Cat Bradley Max Robinson RUBEN DELGADO GIL Justin Ladner Marvin Sandoval Jeffrey Urbanski Clark Messman Radu Milea steve slauson Daniel Frank Anton Samokhvalov Jeremy Bradford Jacob Banta Lance Anderson Jon Harrison Peter Mortimer  Pat Cade Matthew Harrison Wesley Sandoval Mackenzie Mau Jamie Kilcoyne David Goggins Ryan Van Ness Chad Braun Jonathan Lantz TODD DILL Bryan Kerl Adam Monke Nathaniel Orders Tito Nazar Ezra Becker Zachary Allen Kilian Korth Brooks Williams John Olar Brandon Worthington Samantha Wood Carrie Stafford James Schneiderman Bruce Dailey Dustin Moore Jordan Ricks Timothy Barr Tim Finocchio Sacha Devillaz Phil Reutlinger Drew Adams John Muir Max Fulton paul zani Lelis Gonzalez Brandon Ray Steve Orellana Jens Sperlich Alex Cummings matt harvey Steve Pentler Howard Vogt Daniel Arnold Sam Schwaller Aaron Acuna Leonardo Leite Tim Cronk Michael Robbert Robert Stowe Travis Daniels Niki Pardoe ANDREW WICKLESS jason vieth Michael Alfred Kaitlyn Morgan George Ulrich Michael King Dion Leonard Aaron Ellison Joshua Stevens Craig Sylvester Shannon Howell Chris Wright kenneth ebener RANDY MAST Shaun Baker Gary Stotler aaron fearing Nikita Levchenko Kara Diamond-Husmann David Mackey Jonathan Joslyn Paul Tucker Mat Grills Stacey Marion Peter Grenney Jason Romero Francesc Basacoma Matthew Hunsucker Janis Snepste Ondrej Tomek Richie Dart Neil Feldman Benjamin Hauschulz Chris Mead Rod Bolls Emily Messegee Joshua Dickson Hannah Jochem Katrin Silva MELANIE WHITE kenneth erichsen Stephen Rodgers Giles Hedley John Mollenhour Daniela Seyler Walter Handloser Bill Hoffman Alan Smith Cristobal Forno David Kerr III Clayton Shapiro Dan Zolnikov Christine O'Gorman Jeremy Howard Hope Bain Tracy O'Brien Anders Wood John Goble Randy Semrau Jay Hagan Emily Luhrs Cristian Vargas Max Lawler Nic De Beer Jeff Mohrmann Sean McLaughlin Rob Wright Katy Anderson Amar Kuchinad Cory Leppert Matthew Kirkendall Dreama Walton Wade Jarvis Max Forgensi Tom Welker Andrew Flinn Drew McManus Andy Bourke Jeff Zenger Martin Schneekloth R. Sean Churchill Michael Nothem Michael Sylvester Gabriel Hendry Erin Park Lillie Romeiser Zach Allen Lucas Onan Andrew Coleman Luke Pospisil Eric Nelson Robert Wright Brandon Wilmoth Denise McHale javier vilchis Clark Mitchell Caitlin Barr Neal Palles April Kenyon Marc Grabowski Jemil Kassahun Kenneth Biddison Erin Drasler John (Muck) kilpatrick TR Maloney Carey Willson LaMar Jost John Paul Ogden Lance Pearce Pete Kardasis Phil Atkinson Ted Richardson John Music Sonja Brester Robert J Koch Mike Wasson Geoff Hanley Canice Harte BAREFOOT MCDONALD Scott Sundahl James Koch Brandy Taylor Jason Goss Joe Miller Paul Dube Newton McCollum tyler quinn Kayde Anderson Lisa Nelson Andrew Erickson Mihai Serban ROBERT MAZURKIEWICZ helen degennaro Caspar Sprungli Brian Burton Ryan McAfee Trey Simcik Becky Grebosky Donna Provenzano Denzil Jennings Alexandra Mittelman Jeremiah Jenson Sam Stephenson Bruno Furrer Amanda Webb Neil Smith Colleen Mentz Chris Kumm Jason Carey Brad Anderson Ciro Ortega Andrew Gzybowski Madeline Hirschfeld Jeremiah Nemechek Erik Nielsen Moon-Ka LEUNG Jack Repasky Erik Hull Brian Merseal Dave Ballard Kevin Moore Erin Laplander Charles Bybee Walter Edwards Kevin Koncilja Jimmy Daly Brian Passenti Abby Reinholt Evan Weisel Junko Kazukawa Bryan Wisdom Allen Kinsler Trish Holbel Sergio Saenz Robb McLean RAMIRO HECTOR CALDERON  COLOMO Paul Nielsen Chung-Man Tam Tyler Patterson Ted Politte Ethan Pence Todd Pizzi Sean Doran Monica Obsitos Jacey Anderson Karolina Zavisiute Jim Ploof Jack Rawls Aaron Cook Benjamin Corwin Luke Finken Alexa Roop Stephen Brown Brent Heady Kristin Walls Daniel Madalena Hawaiian Shirt Ray Churgovich Nathan Pierce Ben Santerre Harrison Hornung Lexi Miller Chris Klinghagen Chris Minton Jason Steinberg mike yasinski Case

# Figures

In [17]:
# Define variables shared among plotting functions

from plotly import colors

# from . import make_figs as f
import make_figs as f


# DF_SPLIT_INFO = io.load_df_split_info_clean(settings.CLEAN_RACE_DATA_DIR)
# DF_SPLIT_TIMES = io.load_df_split_times_clean(settings.CLEAN_RACE_DATA_DIR)

# colors.sequential.thermal
COLOR_SCALE = colors.sequential.Turbo  # could I just put 'Turbo'?
# NOTE: Could also build this dict manually.
COLORS = {
  split_label: COLOR_SCALE[i+2]
  for i, split_label in enumerate(df_split_info.index)
}

# ['ggplot2', 'seaborn', 'simple_white', 'plotly',
#  'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
#  'ygridoff', 'gridon', 'none']
TEMPLATE = 'plotly_dark'

'plotly_dark'

In [18]:
# Plot split times

split_label = df_split_times.index[4]

# fig = f.make_fig_segment_times(df_split_times, split_label)
fig = f.make_fig(df_split_times, f.make_trace_segment_times, split_label)
fig.update_layout(dict(
  title_text=f'Segment time: into {df_split_info.loc[split_label, "name"]}', 
  xaxis_title='Cumulative time (hours)',
  yaxis_title='Segment time (hours)',
))
fig.update_xaxes(dtick=1.0)
fig.update_yaxes(dtick=1.0)

In [19]:
split_label = df_split_times.index[1]

# fig = make_fig_segment_fractions(df_split_times, df_split_times.index[1])
fig = f.make_fig(df_split_times, f.make_trace_segment_fractions, split_label)
fig.update_layout(dict(
  title_text=f'Segment percentage: into {df_split_info.loc[split_label, "name"]}',
  xaxis_title='Cumulative time (hours)',
  yaxis_title='% total time on course',
))

In [20]:
# fig = make_fig_subplots_times(df_split_times)
fig = f.make_fig_subplots(df_split_times, f.make_trace_segment_times)
fig.update_layout(title_text='Segment time vs. cumulative time')
fig.update_xaxes(dtick=1.0)
fig.update_yaxes(dtick=1.0)